In [12]:
import cv2 as cv
import numpy as np

In [13]:
from scipy.spatial.transform import Rotation
from algs.uniform_sampling import UniformSampling
from manipulated_object import ObjectPosition


def uniform_dataset(n:int, start_pos:tuple[int,int,int]):
    axes = UniformSampling._uniform_det_axes(n**2)
    axes = np.repeat(axes, n, axis=0)
    rots = np.linspace(0, 2 * np.pi, num=n, endpoint=False)
    rots = np.tile(rots, n**2)
    rot_vec = np.expand_dims(rots, axis=-1) * axes
    orients = Rotation.from_rotvec(rot_vec).as_euler("xyz")
    
    dataset = [ObjectPosition(orient, start_pos) for orient in orients]
    return dataset

In [14]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from evaluate.evaluator import Evaluator
from visualize import SearchPlotter
from algs.mealpy_algorithm import MealpyAlgorithm


# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

experiment_folder = "Evals/test2"

In [15]:
init_location = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
# eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]
eval_positions = uniform_dataset(10, init_location)


evaluator = Evaluator(world_viewer, eval_func=eval_funcs.NormXorDiff(obj_depth=1.0, method="mae"))
evaluator.enable_logging(experiment_folder)

In [16]:
from mealpy.physics_based.CDO import OriginalCDO
from mealpy.swarm_based.PSO import OriginalPSO
from mealpy.evolutionary_based.DE import OriginalDE
from mealpy.evolutionary_based.GA import BaseGA
from mealpy.math_based.CircleSA import OriginalCircleSA
# plotter = SearchPlotter()

loss_func = loss_funcs.IOU()

alg = MealpyAlgorithm(sim_viewer, loss_func=loss_func, optimizer=OriginalCDO())
alg2 = MealpyAlgorithm(sim_viewer, loss_func=loss_func, optimizer=OriginalPSO())
alg3 = MealpyAlgorithm(sim_viewer, loss_func=loss_func, optimizer=BaseGA())


alg_config = MealpyAlgorithm.Config(time_limit=10, seed=1)


for alg, config in [(alg3, alg_config)]:
    eval_losses = evaluator.evaluate(alg, config, eval_positions)
    print(f"{type(alg).__name__}: {eval_losses}")

# plotter.close()
world_viewer.close()

Evaluating Algorithm: MealpyAlgorithm
Alg Config: MealpyAlgorithm.Config(time_limit=10, seed=1, run_mode='single', n_workers=None, log_dest='console')
Loss Function: IOU


Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]

2024/03/16 06:52:03 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.
2024/03/16 06:52:04 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: orient_detection, Epoch: 1, Current best: 0.20990099009900987, Global best: 0.20990099009900987, Runtime: 0.37135 seconds
2024/03/16 06:52:04 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: orient_detection, Epoch: 2, Current best: 0.20990099009900987, Global best: 0.20990099009900987, Runtime: 0.36739 seconds
2024/03/16 06:52:04 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: orient_detection, Epoch: 3, Current best: 0.20990099009900987, Global best: 0.20990099009900987, Runtime: 0.34549 seconds
2024/03/16 06:52:05 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: orient_detection, Epoch: 4, Current best: 0.20990099009900987, Global best: 0.20990099009900987, Runtime: 0.39635 seconds
2024/03/16 06:52:05 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: ori

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/guycohen/opt/anaconda3/envs/robo-proj/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/yj/w4mng56133308xgf6891g49r0000gn/T/ipykernel_13733/1634080613.py", line 18, in <module>
    eval_losses = evaluator.evaluate(alg, config, eval_positions)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/evaluate/evaluator.py", line 45, in evaluate
  File "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/algs/mealpy_algorithm.py", line 85, in find_orientation
    best: Agent = self.optimizer.solve(
                  ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/guycohen/opt/anaconda3/envs/robo-proj/lib/python3.11/site-packages/mealpy/optimizer.py", line 229, in solve
    self.initialization()
  File 

In [17]:
from exp_logging import Experiment

exp_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Evals/test2/BaseGA.res"

exp = Experiment.load(exp_path)



In [18]:
exp.eval_loss_list

[0.6825872740761552,
 0.7586278075800078,
 0.691467159621545,
 0.6965623724517046,
 0.6078069103069235,
 0.21220230863339018,
 0.6946651786943939,
 0.6835121432578356,
 0.7357684430850419,
 0.7291328885713299]